# Processing thread for XMM-Newton EPIC observations

## Step 04, tailored for PN Large Window mode

The following is happening here:

1. Sets the folders where the ODF is and where the products will be saved
2. Checks if the necessary SAS files are available: ccf.cif and sets some environmental variables
3. Generates a spectrum per CCD

In [1]:
import os
import subprocess
import sys
import logging
import glob

import numpy as np

from astropy.io import fits

In [2]:
# function to run shell process
# get the SAS version, will be used for the output folder 
def run_command(command):
    try:
        result = subprocess.run([command], shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        retcode = result.returncode
        if retcode < 0:
            print("Warning: \"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
            logging.warning("\"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
        else:
            print("Info: \"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
            logging.info("\"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
    except OSError as e:
        print("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
        logging.error("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
    return retcode

In [3]:
#obsid = "0721010501"
#obsid = "0721010401"
obsid = "0764530101"

home = os.path.expanduser('~')
wdir = f"{home}/IVAN/Cu-line"

if (not os.path.isdir(wdir)):
    print (f"The input working folder {wdir} does not exist.")
    raise FileNotFoundError
#
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s %(message)s',
                    filename=f'{wdir}/{obsid}_proc_step04.log',
                    filemode='w')
# the ODF directory
odfDir = os.path.join(wdir,obsid)
if (not os.path.isdir(odfDir)):
    print (f"The ODF folder {odfDir} does not exist! Cannot continue!")
    logging.error (f"The ODF folder {odfDir} does not exist! Cannot continue!")
    raise FileNotFoundError
#
ppsDir = os.path.join(odfDir,"PN_LW")
#
if (not os.path.isdir(ppsDir)):
    print (f"PPS folder {ppsDir} does not exist. You need to run step02 before running step03")
    logging.error (f"PPS folder {ppsDir} does not exist. You need to run step02 before running step03")
    raise FileNotFoundError
#
os.chdir(ppsDir)

In [4]:
os.environ['SAS_CCFPATH'] = '/xdata/ccf/pub'
os.environ["SAS_ODF"] = odfDir
ccffile = os.path.join(odfDir,"ccf.cif")
if (not os.path.isfile(ccffile)):
    print (f"No ccf.cif file found in ODF folder {odfDir}")
    print ("     ===> This means epic_proc_step01 was not run?")
    logging.error(f"No CCF file ccf.cif found in {odfDir}")
    raise FileNotFoundError
#
os.environ["SAS_CCF"] = ccffile
logging.info(f"Set SAS_ODF to {odfDir}")
logging.info(f"Set SAS_CCF to {ccffile}")
#

In [5]:
# find the event lists
haveEvlist = False
evfind = glob.glob(f"{ppsDir}/*PIEVLI*")
if (len(evfind) >= 1):
    logging.info(f"Found {len(evfind)} event lists")
    # exposures
    xexpo = [os.path.basename(x)[13:17] for x in evfind]
    print ("Event lists for the following exposures are available: ",xexpo)
    haveEvlist = True
else:
    logging.error("Event list not produced, rerun step02")
    raise FileNotFoundError
#
index = 0
evlist = evfind[index]
nexpo = os.path.basename(evlist)[13:17]
print (f"Will process event list {evlist}")
#

Event lists for the following exposures are available:  ['S003']
Will process event list /home/ivaltchanov/IVAN/Cu-line/0764530101/PN_LW/P0764530101PNS003PIEVLI0000.FIT


In [6]:
print ("*** Generating spectra")
#
# will attache a canned response file for PN in LW, single events
#
respfile = f"{home}/IVAN/PN_LW/epn_e3_lw20_sY9_v17.0.rmf"
#
evsel = "(PATTERN==0) && (PAT_SEQ==0) && #XMMEA_EP"
#
# now loop over all 12 CCDs in EPN
#
for i in range(12):
    ccd = f"{i+1:02}"
    print (f"Processing CCD # {ccd}")
    #
    mnk_mask =  f"/xdata/xcaldata/XMM/PN/CTI/mask/refmask_{ccd}_mnk_lmap.fits"
    #
    sel1 = f"mask({mnk_mask},1,1,RAWX,RAWY) && (CCDNR == {i+1})"
    spec_file = f"pn_{nexpo}_{ccd}_all_spec5.fits"
    command = "evselect " + \
        f"table={evlist} energycolumn='PI' withspectrumset=yes" + \
        f" expression='{evsel} && {sel1}'" + \
        " withspecranges=yes specchannelmin=0 specchannelmax=20479" + \
        f" spectrumset={spec_file} spectralbinsize=5"
    status = run_command(command)
    if (status != 0):
        raise Exception
    #
    # adding some keywords needed for XSPEC
    #
    hdu = fits.open(spec_file)
    hdu[1].header['BACKFILE'] = "NONE"
    hdu[1].header['ANCRFILE'] = "NONE"
    hdu[1].header['RESPFILE'] = respfile
    hdu.writeto(spec_file,overwrite=True)
#    #
#    # now loop over RAWY in 20 pixels
#    #
#    for j in np.arange(100,200,20):
#        sel2 = f"(RAWY > {j} && RAWY <= {j+20})"
#        spec_file = f"pn_{nexpo}_{ccd}_{j}_spec5.fits"
#        command = "evselect " + \
#            f"table={evlist} energycolumn='PI' withspectrumset=yes" + \
#            f" expression='{evsel} && {sel1} && {sel2}'" + \
#            " withspecranges=yes specchannelmin=0 specchannelmax=20479" + \
#            f" spectrumset={spec_file} spectralbinsize=5"
#        status = run_command(command)
#        if (status != 0):
#            raise Exception
    # For CCD 4 we add the boresight with RAWY in [180,200]
    if (ccd == "04"):
        print (f"Extracting the boresight spectrum from CCD {ccd}")
        sel2 = f"(RAWY > 180 && RAWY <= 200)"
        spec_file = f"pn_{nexpo}_{ccd}_180_spec5.fits"
        command = "evselect " + \
            f"table={evlist} energycolumn='PI' withspectrumset=yes" + \
            f" expression='{evsel} && {sel1} && {sel2}'" + \
            " withspecranges=yes specchannelmin=0 specchannelmax=20479" + \
            f" spectrumset={spec_file} spectralbinsize=5"
        status = run_command(command)
        if (status != 0):
            raise Exception
        #
        # adding some keywords needed for XSPEC
        #
        hdu = fits.open(spec_file)
        hdu[1].header['BACKFILE'] = "NONE"
        hdu[1].header['ANCRFILE'] = "NONE"
        hdu[1].header['RESPFILE'] = respfile
        hdu.writeto(spec_file,overwrite=True)
#
logging.info ("pn_lw_step04 done")
print ("pn_lw_step04 done")
#

*** Generating spectra
Processing CCD # 01
Info: "evselect table=/home/ivaltchanov/IVAN/Cu-line/0764530101/PN_LW/P0764530101PNS003PIEVLI0000.FIT energycolumn='PI' withspectrumset=yes expression='(PATTERN==0) && (PAT_SEQ==0) && #XMMEA_EP && mask(/xdata/xcaldata/XMM/PN/CTI/mask/refmask_01_mnk_lmap.fits,1,1,RAWX,RAWY) && (CCDNR == 1)' withspecranges=yes specchannelmin=0 specchannelmax=20479 spectrumset=pn_S003_01_all_spec5.fits spectralbinsize=5" returned 0 
 evselect:- Executing (routine): evselect table=/home/ivaltchanov/IVAN/Cu-line/0764530101/PN_LW/P0764530101PNS003PIEVLI0000.FIT filteredset=filtered.fits withfilteredset=no keepfilteroutput=no flagcolumn=EVFLAG flagbit=-1 destruct=yes dssblock='' expression='(PATTERN==0) && (PAT_SEQ==0) && #XMMEA_EP && mask(/xdata/xcaldata/XMM/PN/CTI/mask/refmask_01_mnk_lmap.fits,1,1,RAWX,RAWY) && (CCDNR == 1)' filtertype=expression cleandss=no updateexposure=yes filterexposure=yes writedss=yes blockstocopy='' attributestocopy='' energycolumn=PI zcolu